In [ ]:
# MINE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- [sbert](https://www.sbert.net/)
- [training_overview](https://www.sbert.net/docs/sentence_transformer/training_overview.html)
- [loss_overview](https://www.sbert.net/docs/sentence_transformer/loss_overview.html)
- [training_examples](https://www.sbert.net/docs/sentence_transformer/training/examples.html)
- [matryoshka](https://www.sbert.net/examples/sentence_transformer/training/matryoshka/README.html)
- [adaptive_layer](https://www.sbert.net/examples/sentence_transformer/training/adaptive_layer/README.html)
- [training_with_prompts](https://www.sbert.net/examples/sentence_transformer/training/prompts/README.html)
- [training_with_peft](https://www.sbert.net/examples/sentence_transformer/training/peft/README.html)
- [link text](https://)

In [ ]:
# @title **All needed imports**
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss, AdaptiveLayerLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

## **Adaptive Layer Fine-tuning**

### **Load the Model**

In [ ]:
# This is optional
model_card=SentenceTransformerModelCardData(
    language='en', license="mit",
    model_name="bge-large-en-v1.5"
)

# A must
model = SentenceTransformer(
    "BAAI/bge-large-en-v1.5",
    model_card_data=model_card
)

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# @title **Load your dataset**
dataset = load_dataset("sentence-transformers/all-nli", "triplet")
# train dataset is about 500K
# Will be using a subset of 200K
train_dataset = dataset["train"].select(range(200_000))
validation_dataset = dataset['dev']
test_dataset = dataset['test']

### **Data Inspection**

In [ ]:
# train
dataset.column_names

{'train': ['anchor', 'positive', 'negative'],
 'dev': ['anchor', 'positive', 'negative'],
 'test': ['anchor', 'positive', 'negative']}

In [ ]:
def random_examples_selector(given_dataset):
   from random import randint
   anchor = given_dataset['anchor'][randint(0, len(given_dataset))]
   positive = given_dataset['positive'][randint(0, len(given_dataset))]
   negative = given_dataset['negative'][randint(0, len(given_dataset))]

   print(f"Anchor: {anchor}")
   print(f"Positive: {positive}")
   print(f"Negative: {negative}")

In [ ]:
random_examples_selector(train_dataset)

Anchor: People are riding horses.
Positive: A black and a tan dog.
Negative: A lady in a black shirt and green skirt.


In [ ]:
random_examples_selector(validation_dataset)

Anchor: You'll even be able to consult a traditional herbalist to cure your ailments.
Positive: Finally, progressives are noticing that the best argument for government activism is its success.
Negative: Go at noon to see the desert for the best view.


In [ ]:
random_examples_selector(test_dataset)

Anchor: Please send your most generous gift today.
Positive: The man is walking in a direction.
Negative: A man sits on a wall.


In [ ]:
def examples_selector(given_dataset,index:int):
   anchor = given_dataset['anchor'][index]
   positive = given_dataset['positive'][index]
   negative = given_dataset['negative'][index]

   print(f"Anchor: {anchor}")
   print(f"Positive: {positive}")
   print(f"Negative: {negative}")

In [ ]:
examples_selector(train_dataset,1)

Anchor: Children smiling and waving at camera
Positive: There are children present
Negative: The kids are frowning


In [ ]:
examples_selector(validation_dataset,1)

Anchor: Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink.
Positive: Two kids in numbered jerseys wash their hands.
Negative: Two kids in jackets walk to school.


In [ ]:
examples_selector(test_dataset,1)

Anchor: A woman with a green headscarf, blue shirt and a very big grin.
Positive: The woman is very happy.
Negative: The woman has been shot.


### **Loss Definition**

In [ ]:
base_loss = MultipleNegativesRankingLoss(model)
loss = AdaptiveLayerLoss(model=model,loss=base_loss)

### **Setting up Weights and Biases for Logging**

In [ ]:
import wandb

In [ ]:
wandb.login()

wandb: Currently logged in as: dannyai (dannyai-danny-the-analyst) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import os
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="Embedding-Fine-Tuning-Adaptive-Layer-bge-large-en-v1.5"

# save your trained model checkpoint to wandb
# os.environ["WANDB_LOG_MODEL"]="true" # throws an error, must use 'checkpoint' or 'end'
os.environ["WANDB_LOG_MODEL"]="checkpoint"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

### **Specify Training Arguments**

[Training Arguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments.save_total_limit)

In [ ]:
training_args = SentenceTransformerTrainingArguments(
    # num_train_epochs=1, # full training,
    max_steps=300, # reduced from 600 to 300
    per_device_train_batch_size=5, # reduced from 16 to 5
    per_device_eval_batch_size=5, # reduced from 16 to 5
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    # Some optional tracking and debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    logging_strategy="steps",
    logging_steps=100,
    # logging_first_step=True,
    load_best_model_at_end=True,
    seed = 30,
    output_dir = "outputs",
    run_name="Embedding_Fine_Tuning_Adaptive_Layer_en_v_1_5",
    report_to=["wandb"] # reporting to Weights and biases project
)

In [ ]:
training_args

SentenceTransformerTrainingArguments(output_dir='outputs', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, eval_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=5, per_device_eval_batch_size=5, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=600, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.1, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='outputs/runs/Sep20_14-28-20_48fc117d9b90', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=100, logging_nan_inf_filter=True, save_strategy=<SaveStrategy.STEPS: 'steps'>, sav

### **Evaluator**

- [TripletEvaluator](https://www.sbert.net/docs/package_reference/sentence_transformer/evaluation.html#sentence_transformers.evaluation.TripletEvaluator)
- [SimilarityFunction](https://www.sbert.net/docs/package_reference/sparse_encoder/SparseEncoder.html#sentence_transformers.SimilarityFunction)

In [ ]:
# Validation set is 6K
# Will use 3K
max_samples = 3000
validation_dataset = validation_dataset.select(range(max_samples))
# Will use all
# validation_dataset = validation_dataset
from sentence_transformers.evaluation import SimilarityFunction
# Initialise the evaluator
val_evaluator = TripletEvaluator(
    anchors = validation_dataset['anchor'],
    positives = validation_dataset['positive'],
    negatives = validation_dataset['negative'],
    batch_size=50,
    main_similarity_function=SimilarityFunction.COSINE,
    show_progress_bar=True,
    name='all-nli-val'
)
# WARNING:sentence_transformers.evaluation.TripletEvaluator:The 'main_distance_function' parameter is deprecated.
# Please use 'main_similarity_function' instead. 'main_distance_function' will be removed in a future release.

In [ ]:
# @title **Create an evaluator & evaluate the base model**
val_evaluator(model)
# {'all-nli-val_cosine_accuracy': 0.9548906683921814}

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

{'all-nli-val_cosine_accuracy': 0.9599999785423279}

In [ ]:
# @title **Create a trainer & train**

trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    evaluator=val_evaluator,
    loss=loss
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,All-nli-val Cosine Accuracy
100,0.732600,0.527313,0.952000
200,0.662500,0.595400,0.949000
300,0.608800,0.524985,0.948667
400,0.505300,0.528894,0.955000
500,0.603900,0.510080,0.948333
600,0.592300,0.503658,0.951333


Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

wandb: Adding directory to artifact (outputs/checkpoint-100)... Done. 47.2s


...

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

wandb: Adding directory to artifact (outputs/checkpoint-600)... Done. 56.6s


TrainOutput(global_step=600, training_loss=0.6175700632731119, metrics={'train_runtime': 2968.1222, 'train_samples_per_second': 1.011, 'train_steps_per_second': 0.202, 'total_flos': 0.0, 'train_loss': 0.6175700632731119, 'epoch': 0.015})

In [ ]:
# @title **Evaluate the trained model on the test set**
test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="all-nli-test",
)

In [ ]:
test_evaluator(model)

{'all-nli-test_cosine_accuracy': 0.9527916312217712}

### **Model Saving**

In [ ]:
# @title **Save the trained model**
model.save_pretrained("embedding_model/embedding_fine_tuning_adaptive_layer_bge-large-en-v1.5")

In [ ]:
# @title **Push it to the Hugging Face Hub**
# do not create repo on hugging face
model.push_to_hub("embedding_fine_tuning_adaptive_layer_bge-large-en-v1.5", exist_ok=True)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp34vuvxgq/model.safetensors    :   0%|          | 28.6kB / 1.34GB            

'https://huggingface.co/DannyAI/embedding_fine_tuning_adaptive_layer_bge-large-en-v1.5/commit/d59baac5432a73797b9b1ad44ddaf0f3c6321e3f'